# Connected Agents Tutorial

🤝 **Learn how to create connected agents that work together!**

This tutorial demonstrates:
1. **Part 1: Azure AI Foundry SDK** - Connected agents using built-in tools
2. **Part 2: Semantic Kernel SDK** - Multi-agent orchestration

**Connected agents** allow multiple specialized AI agents to collaborate and share information to solve complex tasks.

---

## 🔧 Setup and Prerequisites

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name

**For Semantic Kernel:**
- `AZURE_OPENAI_ENDPOINT`: Your Azure OpenAI endpoint
- `AZURE_OPENAI_API_KEY`: Your Azure OpenAI API key
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`: Your chat model deployment

In [1]:
# Install required packages
# !pip install azure-ai-agents azure-identity semantic-kernel

import os
import asyncio
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole
from azure.identity import DefaultAzureCredential

print("✅ Packages imported successfully!")

✅ Packages imported successfully!


---

# Part 1: Azure AI Foundry SDK - Connected Agents

The Foundry SDK provides **ConnectedAgentTool** - agents can call other agents as tools.

**Architecture:**
```
Main Agent → Stock Agent (gets Microsoft stock price)
          → Weather Agent (gets Seattle weather)
```

In [16]:
# Create Azure AI Agents client
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

print("🔗 Connected to Azure AI Agents service")

🔗 Connected to Azure AI Agents service


In [17]:
# Create specialized agents
stock_agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="stock_expert",
    instructions="You provide stock prices. For Microsoft, always return $350."
)

weather_agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="weather_expert",
    instructions="You provide weather info. For Seattle, always return 60°F and cloudy."
)

print(f"📈 Stock agent: {stock_agent.id}")
print(f"🌤️ Weather agent: {weather_agent.id}")

📈 Stock agent: asst_Fpe67eo7dnddQq7qUfSDXcIe
🌤️ Weather agent: asst_t6ZC8tTzAPGV7RGJoMrt2Kzo


In [18]:
# Create connected agent tools
stock_tool = ConnectedAgentTool(
    id=stock_agent.id,
    name="stock_expert",
    description="Gets stock prices for companies"
)

weather_tool = ConnectedAgentTool(
    id=weather_agent.id,
    name="weather_expert",
    description="Gets weather information for locations"
)

print("🔧 Connected agent tools created")

🔧 Connected agent tools created


In [19]:
# Create coordinator agent with connected tools
coordinator = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="coordinator",
    instructions="You coordinate with specialist agents to provide comprehensive information.",
    tools=[
        stock_tool.definitions[0],
        weather_tool.definitions[0]
    ]
)

print(f"🎯 Coordinator agent: {coordinator.id}")

🎯 Coordinator agent: asst_PmuFONM8gyBfllthzZrUlamg


In [ ]:
# Test connected agents
# Create conversation thread
thread = agents_client.threads.create()

# Ask coordinator to use both connected agents
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="What's the Microsoft stock price and Seattle weather?"
)

# Process with coordinator (will call connected agents)
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=coordinator.id
)

print(f"✅ Run status: {run.status}")

# Get final response
print("\n🤖 Coordinator Response:")
messages = agents_client.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant" and msg.text_messages:
        print(f"🤖 Agent response: {msg.text_messages[-1].text.value}")
        break

# Keep specialist agents for Part 2, only cleanup coordinator
agents_client.delete_agent(coordinator.id)

print("\n✅ Part 1 complete! Keeping specialist agents for Part 2...")

✅ Run status: RunStatus.COMPLETED

🤖 Coordinator Response:
🤖 Agent response: The current Microsoft stock price is $350, and the weather in Seattle is 60°F and cloudy.

✅ Part 1 complete! Keeping specialist agents for Part 2...


---

# Part 2: Semantic Kernel SDK - AzureAIAgent Plugins

Semantic Kernel provides **AzureAIAgent** to wrap existing Azure AI agents as SK plugins.

**Architecture:**
```
SK Coordinator Agent
    ↓
Stock Plugin (wraps Azure AI stock agent)
Weather Plugin (wraps Azure AI weather agent)
```

In [21]:
# Semantic Kernel imports
from azure.identity.aio import DefaultAzureCredential as AsyncDefaultAzureCredential
from semantic_kernel import Kernel
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function
from typing import Annotated

print("📚 Semantic Kernel imports loaded")

📚 Semantic Kernel imports loaded


In [24]:
# Create Azure OpenAI service for SK coordinator
service = AzureChatCompletion()

print("🔗 Azure OpenAI service configured")

🔗 Azure OpenAI service configured


In [27]:
# Create AzureAIAgent wrappers and SK coordinator
async def create_sk_coordinator():
    endpoint = os.environ["PROJECT_ENDPOINT"]

    async with (
        AsyncDefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds, endpoint=endpoint) as sk_client,
    ):
        print("🔗 Creating SK client and agent wrappers...")
        
        # Wrap existing Azure AI agents as AzureAIAgents
        stock_sk_agent = AzureAIAgent(
            client=sk_client,
            definition=stock_agent  # Use the agent from Part 1
        )
        
        weather_sk_agent = AzureAIAgent(
            client=sk_client,
            definition=weather_agent  # Use the agent from Part 1
        )
        
        print("🎯 Azure AI agents wrapped as SK plugins")
        
        # Create kernel and add plugins
        kernel = Kernel()
        kernel.add_plugin(stock_sk_agent, "stock")
        kernel.add_plugin(weather_sk_agent, "weather")
        
        # Create SK coordinator agent with plugins
        coordinator_sk = ChatCompletionAgent(
            name="SKCoordinator",
            instructions="You coordinate information using available functions. Call stock functions for stock data and weather functions for weather data.",
            service=service,
            kernel=kernel
        )
        
        # Test the coordination
        print("\n🧪 Testing SK coordinator with Azure AI agent plugins...")
        
        user_input = "Get me Microsoft's stock price and Seattle's weather"
        print(f"👤 User: {user_input}")
        
        async for response in coordinator_sk.invoke(user_input):
            print(f"🤖 SK Coordinator: {response.content}")
            final_response = response.content
        
        return final_response

# Run the SK coordination
result = await create_sk_coordinator()
print("\n✅ Semantic Kernel coordination with Azure AI agents complete!")

🔗 Creating SK client and agent wrappers...
🎯 Azure AI agents wrapped as SK plugins

🧪 Testing SK coordinator with Azure AI agent plugins...
👤 User: Get me Microsoft's stock price and Seattle's weather
🤖 SK Coordinator: Here’s the latest information you asked for:

• Microsoft (MSFT) stock price: $350  
• Seattle weather: 60 °F and cloudy
🤖 SK Coordinator: Here’s the latest information you asked for:

• Microsoft (MSFT) stock price: $350  
• Seattle weather: 60 °F and cloudy


ERROR:root:Error while closing connector: ClientConnectionError('Connection lost: SSL shutdown timed out')



✅ Semantic Kernel coordination with Azure AI agents complete!


In [ ]:
# Final cleanup - delete the remaining Azure AI agents
agents_client.delete_agent(stock_agent.id)
agents_client.delete_agent(weather_agent.id)

print("🧹 All Azure AI agents cleaned up")

---

## 🔄 Comparison: Connected Agents vs AzureAI Plugins

| Aspect | Foundry SDK (Connected) | Semantic Kernel (AzureAI Plugins) |
|--------|-------------------------|-----------------------------------|
| **Connection** | Agent calls other agents as tools | Azure AI agents wrapped as SK plugins |
| **Flow** | Tool-based invocation | Plugin function calls |
| **Use Case** | Task delegation | Hybrid SK + Azure AI coordination |
| **Complexity** | Simple, direct calls | SK orchestration with Azure AI power |
| **Context** | Limited to tool parameters | SK context + Azure AI capabilities |

### When to Use Each:

**🔗 Connected Agents (Foundry SDK):**
- Pure Azure AI agent coordination
- Simple task delegation
- Real-time tool usage

**🔌 AzureAI Plugins (Semantic Kernel):**
- Hybrid SK + Azure AI workflows
- Rich SK orchestration features
- Leverage existing Azure AI agents in SK

---

## 🎯 Key Takeaways

✅ **Connected Agents (Foundry SDK):**
- Use `ConnectedAgentTool` for agent-to-agent communication
- Perfect for specialized function delegation
- Built-in tool management

✅ **AzureAI Plugins (Semantic Kernel):**
- Use `AzureAIAgent` to wrap existing Azure AI agents
- Leverage SK's rich plugin system
- Combine Azure AI power with SK orchestration

✅ **Both Approaches:**
- Enable agent specialization
- Improve task decomposition
- Scale complex AI solutions
- Reuse existing Azure AI agents

**🚀 Next Steps:**
- Experiment with different plugin patterns
- Try combining multiple Azure AI agents in SK
- Explore hybrid workflows with both SDKs

**Happy agent orchestrating!** 🎉